# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [63]:
#Libraries
import os
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [64]:
# load dataset(messages)
df_messages = pd.read_csv("dataset/messages.csv")
# check number of rows and columns if dataframe
print("Number of rows:", df_messages.shape[0])
print("Number of columns:", df_messages.shape[1])
df_messages.head()

Number of rows: 26248
Number of columns: 4


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [65]:
# load categories dataset
df_categories = pd.read_csv("dataset/categories.csv")
# check number of rows and columns if dataframe
print("Number of rows:", df_categories.shape[0])
print("Number of columns:", df_categories.shape[1])
df_categories.head()

Number of rows: 26248
Number of columns: 2


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [66]:
# merge datasets
df = df_messages.merge(df_categories, how ='outer', on =['id'])
print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 26386
Number of columns: 5


,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [67]:
def preprocess_categories(df):
    """
    Preprocess the 'categories' column of a dataframe by splitting it into separate columns and extracting category column names.

    Parameters:
    - df: DataFrame
        Input dataframe containing a 'categories' column.

    Returns:
    - categories: DataFrame
        DataFrame containing the split values from the 'categories' column.
    - category_colnames: Series
        Series containing the extracted category column names.
    """
    # Split the 'categories' column of the dataframe 'df' by semicolons into separate columns, expanding them horizontally.
    # This creates a dataframe 'categories' with the split values.
    categories = df['categories'].str.split(';', expand=True)

    # Select the first row of the 'categories' dataframe and assign it to the variable 'row'.
    row = categories.head(1)

    # Extract the category column names from the first row by removing the last two characters from each value.
    # These values represent the category labels without the numeric suffixes.
    category_colnames = row.applymap(lambda x: x[:-2]).iloc[0,:]

    return categories, category_colnames

In [68]:
# Call the preprocess_categories function and store the return values
categories, category_colnames = preprocess_categories(df)

# Rename the columns of `categories`
categories.columns = category_colnames

# Display the first few rows of the updated `categories` DataFrame
categories.head()

C:\Users\bonzoi\AppData\Local\Temp\ipykernel_23080\4017132380.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  category_colnames = row.applymap(lambda x: x[:-2]).iloc[0,:]


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [69]:
def convert_categories_to_numeric(categories):
    """
    Convert categorical values in each column of the 'categories' dataframe to numeric format.

    Parameters:
    - categories: DataFrame
        DataFrame containing categorical values in its columns.

    Returns:
    - categories: DataFrame
        DataFrame with categorical values converted to numeric format.
    """
    # Iterate over each column in the 'categories' dataframe
    for column in categories:
        # Set each value to be the last character of the string
        categories[column] = categories[column].astype(str).str[-1]
        
        # Convert column from string to numeric
        categories[column] = categories[column].astype(int)
    
    return categories

# Apply the function to the 'categories' dataframe
categories = convert_categories_to_numeric(categories)

# Display the first few rows of the updated 'categories' dataframe
categories.head()


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [70]:
# drop the original categories column from `df`
df.drop('categories', axis=1, inplace=True)

df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [71]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [72]:
def check_and_remove_duplicates(df):
    """
    Check for duplicates in a DataFrame and remove them.
    
    Parameters:
        df (pandas.DataFrame): The DataFrame to check for duplicates.
    
    Returns:
        None
    """
    # Check number of duplicates in the DataFrame
    num_duplicates_before = df.duplicated().sum()

    # Drop duplicates from the DataFrame
    df.drop_duplicates(inplace=True)

    # Check number of duplicates after dropping
    num_duplicates_after = df.duplicated().sum()

    # Output the results
    print(f"Number of duplicates before: {num_duplicates_before}")
    print(f"Number of duplicates after: {num_duplicates_after}")

# Example usage:
check_and_remove_duplicates(df)


Number of duplicates before: 170
Number of duplicates after: 0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [73]:
# Define the directory where you want to save the database file
folder_path = "Dataset"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Specify the file path including the folder
db_path = os.path.join(folder_path, "WB_disaster_Database.db")

# Create a SQLAlchemy engine to connect to the SQLite database
engine = create_engine(f'sqlite:///{db_path}')

# Save the DataFrame to the SQLite database
df.to_sql('WB_disaster_messages', engine, index=False, if_exists='replace')

26216

### 8. Use this notebook to complete `process_data.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `process_data.py` in the classroom on the `Project Workspace IDE` coming later.